
# Australian Threatened Plants (TSX): Data Wrangling and MySQL Ingestion

Name: Zihan

### Step 1: Extract, Transform Species and Location Metadata

This step's code performs the following operations:
1.  Use `pandas` library to read original aggregated dataset `06_tsx-aggregated-data-dataset_for_aus_plants.csv`.
2.  Select key columns related to species classification, conservation status, and geographic location based on specified column list.
3.  Convert numeric values (1 and 0) in `NationalPriorityTaxa` column to more readable text ('Yes' and 'No').
4.  Save extracted and processed subset data to new CSV file `Table16_TSX_SpeciesMonitoringTable.csv` in `02_wrangled_data` directory.

In [ ]:
import pandas as pd
import os

# --- 1. Define file paths ---
# Original data file path
source_file_path = '01_raw_data/06_tsx-aggregated-data-dataset_for_aus_plants.csv'

# Output file path
output_file_path = '02_wrangled_data/Table16_TSX_SpeciesMonitoringTable.csv'

# --- 2. Define list of columns to extract ---
# Note: 'ID' in original file is uppercase, maintain consistency here
columns_to_extract = [
    'ID',
    'Binomial',
    'CommonName',
    'FamilyCommonName',
    'Class',
    'Order',
    'Family',
    'Genus',
    'Species',
    'Subspecies',
    'FunctionalGroup',
    'EPBCStatus',
    'IUCNStatus',
    'MaxStatus',
    'NationalPriorityTaxa',
    'State',
    'Region',
    'RegionCentroidLatitude',
    'RegionCentroidLongitude'
]

# --- 3. Ensure output directory exists ---
# Get directory containing output file
output_dir = os.path.dirname(output_file_path)
# Create directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# --- 4. Execute data extraction, transformation and saving ---
try:
    # Read original CSV file
    print(f"Reading original file: {source_file_path}")
    df_raw = pd.read_csv(source_file_path)
    
    # Select specified columns from DataFrame
    df_subset = df_raw[columns_to_extract].copy() # Use .copy() to avoid SettingWithCopyWarning
    
    # *** New step: Convert 'NationalPriorityTaxa' column ***
    # Create mapping dictionary
    mapping = {1: 'Yes', 0: 'No'}
    # Apply mapping
    df_subset['NationalPriorityTaxa'] = df_subset['NationalPriorityTaxa'].map(mapping)
    print("Converted 'NationalPriorityTaxa' column from 1/0 to Yes/No.")
    
    # Save extracted data to new CSV file without index column
    df_subset.to_csv(output_file_path, index=False)
    
    print("-" * 50)
    print(f"Data successfully extracted and transformed!")
    print(f"New file saved to: {output_file_path}")
    print(f"New file contains {df_subset.shape[0]} rows and {df_subset.shape[1]} columns.")
    print("-" * 50)
    # Display first few rows of converted column for verification
    print("Preview of converted 'NationalPriorityTaxa' column:")
    print(df_subset['NationalPriorityTaxa'].head())


except FileNotFoundError:
    print(f"Error: Original file not found. Please confirm file path is correct: {source_file_path}")
except KeyError as e:
    print(f"Error: Original file missing one or more specified columns. Missing column: {e}")
except Exception as e:
    print(f"Unknown error occurred: {e}")

正在读取原始文件: 01_raw_data/06_tsx-aggregated-data-dataset_for_aus_plants.csv
已将 'NationalPriorityTaxa' 列从 1/0 转换为 Yes/No。
--------------------------------------------------
成功提取并转换数据！
新的文件已保存至: 02_wrangled_data/Table16_TSX_SpeciesMonitoringTable.csv
新文件包含 937 行 和 19 列。
--------------------------------------------------
转换后 'NationalPriorityTaxa' 列的预览:
0    No
1    No
2    No
3    No
4    No
Name: NationalPriorityTaxa, dtype: object


### Step 1 - Load Processed Data for Verification
This step loads the cleaned and transformed CSV file from previous step and performs quick preview to ensure data is correct before uploading.

In [ ]:
import pandas as pd

# Define path to our processed data file
wrangled_file_path = '02_wrangled_data/Table16_TSX_SpeciesMonitoringTable.csv'

# Read data into DataFrame
try:
    df = pd.read_csv(wrangled_file_path)
    print("✅ Processed CSV file loaded successfully!")
    print("\n" + "="*50 + "\n")
    
    print("Data overview:")
    df.info()
    
    print("\nData preview (first 5 rows):")
    print(df.head())

except FileNotFoundError:
    print(f"❌ Error: File not found. Please check path '{wrangled_file_path}' is correct.")
except Exception as e:
    print(f"❌ Error occurred during processing: {e}")

✅ 已处理的CSV文件加载成功！


数据信息概览：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       937 non-null    int64  
 1   Binomial                 937 non-null    object 
 2   CommonName               850 non-null    object 
 3   FamilyCommonName         0 non-null      float64
 4   Class                    0 non-null      float64
 5   Order                    937 non-null    object 
 6   Family                   937 non-null    object 
 7   Genus                    937 non-null    object 
 8   Species                  937 non-null    object 
 9   Subspecies               76 non-null     object 
 10  FunctionalGroup          937 non-null    object 
 11  EPBCStatus               932 non-null    object 
 12  IUCNStatus               149 non-null    object 
 13  MaxStatus                937 non-null    object 
 14 

### Step 2 - Create Database Table Structure for `Table16_TSX_SpeciesMonitoringTable`
This step connects to your MySQL database, drops any existing old table, then creates new, properly structured table based on specified columns and data types.

In [ ]:
import mysql.connector
from mysql.connector import Error

# Database connection configuration (consistent with what you provided)
db_config = {
    'host': 'database-plantx.cqz06uycysiz.us-east-1.rds.amazonaws.com',
    'user': 'zihan',
    'password': '2002317Yzh12138.',
    'database': 'FIT5120_PlantX_Database',
    'allow_local_infile': True,
    'use_pure': True,
    'charset': 'utf8mb4'
}

# --- Task: Create table structure ---
print("--- [Task 1/4] Creating table 'Table16_TSX_SpeciesMonitoringTable'... ---")
try:
    connection = mysql.connector.connect(**db_config)
    if connection.is_connected():
        print("✅ Successfully connected to MySQL server")
        cursor = connection.cursor()

        table_name = "Table16_TSX_SpeciesMonitoringTable"
        print(f"Dropping old table '{table_name}' (if exists)...")
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
        print("Old table dropped.")

        # SQL statement to create table based on your data types
        create_table_16 = f"""
        CREATE TABLE {table_name} (
            ID INT PRIMARY KEY,
            Binomial TEXT,
            CommonName TEXT,
            FamilyCommonName TEXT,
            `Class` TEXT,
            `Order` TEXT,
            Family TEXT,
            Genus TEXT,
            Species TEXT,
            Subspecies TEXT,
            FunctionalGroup TEXT,
            EPBCStatus TEXT,
            IUCNStatus TEXT,
            MaxStatus TEXT,
            NationalPriorityTaxa TEXT,
            State TEXT,
            Region TEXT,
            RegionCentroidLatitude DOUBLE,
            RegionCentroidLongitude DOUBLE,
            -- Add spatial data column with temporary default value to support LOAD DATA
            coords POINT NOT NULL DEFAULT (POINT(0,0)),
            SPATIAL INDEX(coords)
        ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
        """
        # Note: `Class` and `Order` are SQL reserved keywords, so enclose them in backticks ``
        
        cursor.execute(create_table_16)
        connection.commit()
        print(f"✅ Table '{table_name}' structure created successfully.")

except Error as e:
    print(f"❌ Error creating table structure: {e}")

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("🔌 MySQL connection for table creation closed.\n")

--- [任务 1/4] 正在创建表 'Table16_TSX_SpeciesMonitoringTable'... ---
✅ 成功连接到MySQL服务器
正在删除旧表 'Table16_TSX_SpeciesMonitoringTable' (如果存在)...
旧表已删除。
✅ 表 'Table16_TSX_SpeciesMonitoringTable' 的结构创建成功。
🔌 已关闭用于创建表结构的MySQL连接。



### Step 3 - Import Data into `Table16_TSX_SpeciesMonitoringTable`
Use `LOAD DATA LOCAL INFILE` to efficiently bulk import CSV data into newly created database table.

In [ ]:
try:
    # Re-establish connection for data import
    connection = mysql.connector.connect(**db_config)
    if connection.is_connected():
        print("✅ Successfully connected to MySQL server, preparing to import Table16 data.")
        cursor = connection.cursor()
        
        table_name = "Table16_TSX_SpeciesMonitoringTable"
        csv_path = '02_wrangled_data/Table16_TSX_SpeciesMonitoringTable.csv'

        # Define query to load data
        load_data_query_16 = f"""
        LOAD DATA LOCAL INFILE '{csv_path}'
        INTO TABLE {table_name}
        CHARACTER SET utf8mb4
        FIELDS TERMINATED BY ','
        OPTIONALLY ENCLOSED BY '"'
        LINES TERMINATED BY '\\r\\n'
        IGNORE 1 LINES
        (
            ID, Binomial, CommonName, FamilyCommonName, `Class`, `Order`, Family,
            Genus, Species, Subspecies, FunctionalGroup, EPBCStatus, IUCNStatus,
            MaxStatus, NationalPriorityTaxa, State, Region,
            RegionCentroidLatitude, RegionCentroidLongitude
        );
        """

        cursor.execute(load_data_query_16)
        connection.commit()
        print(f"✅ Table16 data imported successfully! Rows affected: {cursor.rowcount}.")

except Error as e:
    print(f"❌ Error importing Table16 data: {e}")

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("🔌 MySQL connection for Table16 data import closed.")

✅ 成功连接到MySQL服务器，准备导入Table16数据。
✅ Table16数据导入成功！影响行数: 937。
🔌 已关闭用于导入Table16数据的MySQL连接。


### Step 4 - Populate Spatial Data Column (coords)
This step uses imported latitude/longitude data to populate `coords` spatial column.

In [ ]:
try:
    # Re-establish connection to update spatial column
    connection = mysql.connector.connect(**db_config)
    if connection.is_connected():
        print("✅ Successfully connected to MySQL server, preparing to update spatial column.")
        cursor = connection.cursor()

        table_name = "Table16_TSX_SpeciesMonitoringTable"
        
        # SQL statement: Populate 'coords' column from lat/lon (POINT function uses longitude, latitude order)
        update_spatial_column = f"""
        UPDATE {table_name}
        SET coords = POINT(RegionCentroidLongitude, RegionCentroidLatitude)
        WHERE RegionCentroidLongitude IS NOT NULL AND RegionCentroidLatitude IS NOT NULL;
        """

        cursor.execute(update_spatial_column)
        connection.commit()
        print(f"✅ Spatial column 'coords' populated successfully! Updated rows: {cursor.rowcount}.")

except Error as e:
    print(f"❌ Error updating spatial column: {e}")

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("🔌 MySQL connection for spatial column update closed.")

✅ 成功连接到MySQL服务器，准备更新空间列。
✅ 空间列 'coords' 填充成功！更新行数: 937。
🔌 已关闭用于更新空间列的MySQL连接。


### Step 5 - Verify Imported Data
Finally, we connect to database to query total row count and preview first few rows to ensure all content imported correctly, especially `coords` column.

In [ ]:
try:
    connection = mysql.connector.connect(**db_config)
    if connection.is_connected():
        print("\n✅ Successfully connected to MySQL server, preparing to verify Table16 data.")
        cursor = connection.cursor()
        
        table_name = "Table16_TSX_SpeciesMonitoringTable"

        # Get total row count in table
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        row_count = cursor.fetchone()[0]
        print(f"📊 Table '{table_name}' currently contains {row_count} rows.")

        # Get and print first 5 rows for preview
        print(f"\n--- '{table_name}' table first 5 rows preview ---")
        # Use ST_AsText() to display POINT data in readable format
        query = f"""
        SELECT ID, Binomial, Region, 
               RegionCentroidLatitude, RegionCentroidLongitude, ST_AsText(coords) 
        FROM {table_name} 
        LIMIT 5
        """
        cursor.execute(query)
        rows = cursor.fetchall()
        
        # Print table header
        print(f"{'ID':<6} | {'Binomial':<45} | {'Region':<20} | {'coords'}")
        print("-" * 100)
        for row in rows:
            # Format output (ID, Binomial, Region, ST_AsText(coords))
            print(f"{row[0]:<6} | {row[1]:<45} | {row[2]:<20} | {row[5]}")

except Error as e:
    print(f"❌ Error verifying Table16 data: {e}")

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("\n🔌 MySQL connection for Table16 data verification closed.")


✅ 成功连接到MySQL服务器，准备验证Table16数据。
📊 表 'Table16_TSX_SpeciesMonitoringTable' 当前包含 937 行。

--- 'Table16_TSX_SpeciesMonitoringTable' 表前5行数据预览 ---
ID     | Binomial                                      | Region               | coords
----------------------------------------------------------------------------------------------------
4714   | Banksia_ionthocarpa_subsp_chrysophoenix       | Katanning            | POINT(117.1792762 -32.41332852)
4715   | Banksia_ionthocarpa_subsp_chrysophoenix       | Katanning            | POINT(117.1792762 -32.41332852)
4716   | Banksia_ionthocarpa_subsp_chrysophoenix       | Katanning            | POINT(117.1792762 -32.41332852)
4717   | Banksia_ionthocarpa_subsp_chrysophoenix       | Katanning            | POINT(117.1792762 -32.41332852)
4718   | Banksia_ionthocarpa_subsp_ionthocarpa         | Fitzgerald           | POINT(119.1504387 -34.23074081)

🔌 已关闭用于验证Table16数据的MySQL连接。


### Step 6 - Spatial Query Example: Find Plant Monitoring Areas Near Melbourne CBD
Following SQL query demonstrates how to use `coords` column and spatial index to efficiently find all plant monitoring areas within specified radius of given coordinate point (Melbourne CBD).

#### Chinese Version
```sql
-- 设置目标坐标（墨尔本市中心：经度144.9631, 纬度-37.8136）和搜索半径（100公里）
SET @center_point = POINT(144.9631, -37.8136);
SET @radius_meters = 100 * 1000; -- 100公里转换为米

SELECT
    ID,
    Binomial,
    CommonName,
    Region,
    State,
    -- 使用 ST_Distance_Sphere 计算精确的球面距离（单位：米）
    -- 并将其转换为公里以便阅读
    (ST_Distance_Sphere(coords, @center_point) / 1000) AS distance_in_km
FROM
    Table16_TSX_SpeciesMonitoringTable
WHERE
    -- 在 WHERE 子句中直接使用该函数进行高效筛选
    -- MySQL会利用空间索引来优化这个查询，避免全表扫描
    ST_Distance_Sphere(coords, @center_point) <= @radius_meters
ORDER BY
    distance_in_km ASC; -- 按距离从近到远排序
```

#### English Version
```sql
-- Set the target coordinates (Melbourne CBD: Longitude 144.9631, Latitude -37.8136) and search radius (100 km)
SET @center_point = POINT(144.9631, -37.8136);
SET @radius_meters = 100 * 1000; -- Convert 100 km to meters

SELECT
    -- Select key identifiers for the monitoring site
    ID,
    Binomial,
    CommonName,
    Region,
    State,

    -- Calculate the great-circle distance on a sphere, returning the result in meters.
    -- Then, convert it to kilometers for better readability.
    (ST_Distance_Sphere(coords, @center_point) / 1000) AS distance_in_km
FROM
    -- Specify the table containing the threatened species monitoring data.
    Table16_TSX_SpeciesMonitoringTable
WHERE
    -- Filter results to include only records within the specified radius of the central point.
    -- This operation is highly efficient as it leverages the SPATIAL INDEX on the 'coords' column,
    -- preventing a full table scan.
    ST_Distance_Sphere(coords, @center_point) <= @radius_meters
ORDER BY
    -- Sort the results by distance, from nearest to farthest.
    distance_in_km ASC;